In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

Saving phase1.csv to phase1.csv


In [10]:
df = pd.read_csv("FIT VSPP 2565-2567.csv")
fuel_types = df['เชื้อเพลิง'].dropna().unique()
print(fuel_types)

['ชีวมวล' 'ชีวภาพ' 'พลังงานแสงอาทิตย์' 'ขยะ']


In [5]:
import pandas as pd
df = pd.read_csv("FIT VSPP 2565-2567.csv")
df_sorted = df.sort_values(by='id')
df = df[df['year'] != 2568]
df = df.drop(columns=['kwhp', 'kwhop','year','เขต'])
print(df_sorted.head())

      id    เขต เชื้อเพลิง  mwตามสัญญา  mwติดตั้ง  year  month  \
3719  24  กฟฉ.2     ชีวมวล         7.5        9.9  2565      1   
3736  24  กฟฉ.2     ชีวมวล         7.5        9.9  2566      5   
3735  24  กฟฉ.2     ชีวมวล         7.5        9.9  2565      5   
3734  24  กฟฉ.2     ชีวมวล         7.5        9.9  2568      4   
3733  24  กฟฉ.2     ชีวมวล         7.5        9.9  2567      4   

                  kwhp             kwhop               kWH  
3719   \t1,919,030.00    \t3,397,596.00    \t5,316,626.00   
3736   \t1,996,333.00    \t3,492,659.00    \t5,488,992.00   
3735   \t1,989,695.00    \t3,355,262.00    \t5,344,957.00   
3734   \t1,698,339.00    \t3,100,989.00    \t4,799,328.00   
3733   \t1,905,777.00    \t3,223,713.00    \t5,129,490.00   


In [ ]:
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)
df_sorted = df.sort_values(by='id')
df = df[df['year'] != 2568]
df = df.drop(columns=['kwhp', 'kwhop','year','เขต'])
print(df_sorted.head())

      id    เขต เชื้อเพลิง  mwตามสัญญา  mwติดตั้ง  year  month  \
3722  24  กฟฉ.2     ชีวมวล         7.5        9.9  2568      1   
3723  24  กฟฉ.2     ชีวมวล         7.5        9.9  2565      2   
3738  24  กฟฉ.2     ชีวมวล         7.5        9.9  2568      5   
3737  24  กฟฉ.2     ชีวมวล         7.5        9.9  2567      5   
3740  24  กฟฉ.2     ชีวมวล         7.5        9.9  2566      6   

                  kwhp             kwhop               kWH  
3722   \t1,999,919.00    \t3,177,429.00    \t5,177,348.00   
3723   \t1,788,718.00    \t3,118,846.00    \t4,907,564.00   
3738   \t1,973,069.00    \t3,325,298.00    \t5,298,367.00   
3737   \t1,929,538.00    \t3,320,202.00    \t5,249,740.00   
3740   \t2,094,797.00    \t2,961,607.00    \t5,056,404.00   


In [6]:
import numpy as np
import pandas as pd

df['kWH'] = df['kWH'].astype(str).str.replace(',', '').str.strip()
df['kWH'] = df['kWH'].replace(['-', '', ' ', 'nan', '<NA>'], np.nan)
df['kWH'] = df['kWH'].astype(float).fillna(0)
df['AVG_kWH'] = df.groupby(['id', 'month'])['kWH'].transform('mean')
df['kWH_zero'] = (df['kWH'] == 0).astype(int)
missing_rate_df = df.groupby('id')['kWH_zero'].mean().reset_index(name='missing_rate_kWH')
missing_rate_df['missing_rate_kWH'] = (missing_rate_df['missing_rate_kWH'] * 100).round(2)
aggregated_df = df.groupby(['id', 'month'], as_index=False).apply(lambda g: g.iloc[0]).reset_index(drop=True)
aggregated_df = aggregated_df.fillna(0)
aggregated_df['AVG_kWH'] = aggregated_df['AVG_kWH'].round(2)
aggregated_df = aggregated_df.merge(missing_rate_df, on='id', how='left')
print(aggregated_df.drop(columns=['kWH', 'kWH_zero']).head())

   id เชื้อเพลิง  mwตามสัญญา  mwติดตั้ง  month     AVG_kWH  missing_rate_kWH
0  24     ชีวมวล         7.5        9.9      1  5371329.67               0.0
1  24     ชีวมวล         7.5        9.9      2  4880695.67               0.0
2  24     ชีวมวล         7.5        9.9      3  5345938.00               0.0
3  24     ชีวมวล         7.5        9.9      4  5026477.33               0.0
4  24     ชีวมวล         7.5        9.9      5  5361229.67               0.0


/var/folders/r9/8xt0jbx52z129ygv1trvfj200000gn/T/ipykernel_15229/3445140248.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_df = df.groupby(['id', 'month'], as_index=False).apply(lambda g: g.iloc[0]).reset_index(drop=True)


In [7]:
days_per_month = {
    1: 31, 2: 28, 3: 31, 4: 30,
    5: 31, 6: 30, 7: 31, 8: 31,
    9: 30, 10: 31, 11: 30, 12: 31
}
aggregated_df['PF100'] = aggregated_df['month'].map(days_per_month) * aggregated_df['mwตามสัญญา'] * 24000
print(aggregated_df[['id', 'month', 'mwตามสัญญา', 'PF100']].head())

   id  month  mwตามสัญญา      PF100
0  24      1         7.5  5580000.0
1  24      2         7.5  5040000.0
2  24      3         7.5  5580000.0
3  24      4         7.5  5400000.0
4  24      5         7.5  5580000.0


In [8]:
aggregated_df['AVG_kWH'] = aggregated_df['AVG_kWH'].replace(0, np.nan)
aggregated_df['efficiency'] = (aggregated_df['AVG_kWH']/aggregated_df['PF100'] ) * 100
aggregated_df['efficiency'] = aggregated_df['efficiency'].fillna(0)
aggregated_df['efficiency'] = aggregated_df['efficiency'].round(2)
print(aggregated_df[['id', 'month', 'PF100', 'AVG_kWH', 'efficiency']].head())

   id  month      PF100     AVG_kWH  efficiency
0  24      1  5580000.0  5371329.67       96.26
1  24      2  5040000.0  4880695.67       96.84
2  24      3  5580000.0  5345938.00       95.81
3  24      4  5400000.0  5026477.33       93.08
4  24      5  5580000.0  5361229.67       96.08


In [9]:
aggregated_df.drop(columns=['kWH','kWH_zero']).to_csv('1.csv', index=False)